In [2]:
import pandas as pd
import re
import math
import numpy as np

## Preprocessing

In [3]:
shanghaiDataset = "https://raw.githubusercontent.com/riiduan/ProgettoFondamentiDiInformatica/main/data/shanghai.csv"
timesDataset = "https://raw.githubusercontent.com/riiduan/ProgettoFondamentiDiInformatica/main/data/times.csv"
worldDataset="https://raw.githubusercontent.com/riiduan/ProgettoFondamentiDiInformatica/main/data/world.csv"
educational_attainment_supplementary_data ="https://raw.githubusercontent.com/riiduan/ProgettoFondamentiDiInformatica/main/data/educational_attainment_supplementary_data.csv"

In [4]:
#funzione che legge file csv e restituisce pandas dataframe
def readDataSet(url , sep =',' , encoding='utf8',header='infer'):
    return pd.read_csv(url,sep=sep ,header=header,encoding=encoding)

In [5]:
# leggo i dataset
shanghai_df=readDataSet(shanghaiDataset)
times_df=readDataSet(timesDataset)
world_df=readDataSet(worldDataset)
world_df

,Unnamed: 0,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,2195,996,University of the Algarve,Portugal,7,367,567,218,926,845,812,969.0,816,44.03,2015
2196,2196,997,Alexandria University,Egypt,4,236,566,218,997,908,645,981.0,871,44.03,2015
2197,2197,998,Federal University of Ceará,Brazil,18,367,549,218,830,823,812,975.0,824,44.03,2015
2198,2198,999,University of A Coruña,Spain,40,367,567,218,886,974,812,975.0,651,44.02,2015


In [ ]:
# tolgo le rige dove nome di universita è null

index =shanghai_df[shanghai_df['university_name'].isnull()==True]
shanghai_df.drop(index.index,inplace = True)
shanghai_df.reset_index(drop=True, inplace=True)

In [ ]:
# rinomino institutution in "university_name" per averla uguale come i altri dataframe

world_df.rename(columns={'institution':'university_name'},inplace=True)

In [ ]:
shanghai_df.info()

#### controllo le informazioni riguarda ogni dataframe per avere un visione di quanto rige ci sono e i tipo di colone

In [ ]:
times_df.info()

In [ ]:
world_df.info()

## 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [ ]:
'''
questo funzione prende come input il dataframe,  il flag ,e di default nome di universita ,anno,
e in base se è max faccio groupby nome di universita e in base a anno prendo idmax altrimenti idxmin
 

'''
def extract_dataset_min_max(dataframe, flag ,university_name='university_name',year='year' ):
    '''
    find most recent or least recent of data
    @dataframe : dataframe in input
    @university_name  : column university name 
    @year : column year
    @index : max or min
    
    '''
    dataset = pd.DataFrame()
    if(flag == 'max'):
          dataset = dataframe.iloc[times_df.groupby(university_name)[year].idxmax()].copy()
            
    else: dataset = dataframe.iloc[times_df.groupby(university_name)[year].idxmin()].copy()
    return dataset

In [ ]:
times_most_recent_data = extract_dataset_min_max(times_df,'max')

In [ ]:
times_least_recent_data = extract_dataset_min_max(times_df,'min')

In [ ]:
times_most_recent_data[['world_rank','university_name','year']].head(10)

In [ ]:
times_least_recent_data[['world_rank','university_name','year']].head(10)


## 2. For each university, compute the improvement in income between the least recent and the most recent data points

nota : la colona "income" contiene valori "-" , percui prosego prima alla pulizia e poi lo converto in float

In [ ]:
'''
In questa funzione filtro le rige dove la colona income coniente il charatere '-' , prendo tre colone che mi intressano
e converto la collona income in tipo float
'''
def clean_column_income(dataset):
    dataset=dataset[dataset.income.str.contains('-') ==False][['university_name','income','year']].copy()
    dataset.income=dataset.income.astype(float)
    return dataset
            

In [ ]:
times_least_recent_data_clean = clean_column_income(times_least_recent_data)

In [ ]:
times_most_recent_data_clean= clean_column_income(times_most_recent_data)

Unisco i due dataset tramite la colona "university_name"

In [ ]:
union_most_and_least_data =pd.merge(times_least_recent_data_clean,times_most_recent_data_clean,on='university_name')

rinomino le colone 

In [ ]:
union_most_and_least_data.rename(columns={'income_x':'income_level_least_recent','year_x':'year_least_recent','income_y':'income_level_most_recent','year_y':'year_most_recent'},inplace=True)
union_most_and_least_data

##### nota : alcuni universita hanno come "year_most_recent" e "year_most_recent" uguale percui non ha senso tenerli

In [ ]:
union_most_and_least_data[union_most_and_least_data['year_least_recent']==union_most_and_least_data['year_most_recent']].head(100)

elimino queste colone

In [ ]:
union_most_and_least_data.drop(union_most_and_least_data[union_most_and_least_data['year_least_recent']==union_most_and_least_data['year_most_recent']].index,inplace=True)

calcolo la diferenza

In [ ]:
union_most_and_least_data["diference_income"] = union_most_and_least_data.income_level_most_recent - union_most_and_least_data.income_level_least_recent

creo una colona "improvement" che mi indica UP o Down in base se la income è positiva o negativa

In [ ]:
union_most_and_least_data["improvement"] = np.where(union_most_and_least_data["diference_income"] >= 0, "UP", "Down")

In [ ]:
union_most_and_least_data.head(5)

## 3. Find the university with the largest increase computed in the previous point

In [ ]:
# filtro la colona diference_income dal dataset sopra che ha valore piu alto
union_most_and_least_data[union_most_and_least_data.diference_income ==union_most_and_least_data.diference_income.max()]

## 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

#### preprocessing 

nota: dataset shanghai_df , times_df ha alcuni valori espressi in range

In [ ]:
shanghai_df[shanghai_df['world_rank'].str.contains('-')==True]['world_rank'].head(3)

nota : time_df porta con se alcuni valri con segno '=' davanti, che significa valore uguale, percui se lo tolgo non perde significato

In [ ]:
times_df[times_df['world_rank'].str.contains('=')==True]['world_rank'].head(3)

In [ ]:
times_df['world_rank']=times_df['world_rank'].str.replace('=','')

nota: world_df sembra ok , non ha valori in range e valori strani

In [ ]:
world_df['world_rank'].dtype

 ### Dati più recenti per le tabelle 'shanghai_ranking', 'times_ranking' e 'world_ranking

Shanghai

In [ ]:
shanghai_most_recent_data = extract_dataset_min_max(shanghai_df,'max')

In [ ]:
shanghai_most_recent_data.info()

Times

In [ ]:
times_most_recent_data = extract_dataset_min_max(times_df,'max')

In [ ]:
times_most_recent_data.info()

World  

In [ ]:
world_df

In [ ]:
world_df

In [ ]:
world_most_recent_data = extract_dataset_min_max(world_df,'max','university_name','year')

In [ ]:
world_most_recent_data.info()


### Creo nuova colona con nomi di universita puliti (spazi,virgole,tutti minuscoli ect) cosi combaciano

In [ ]:
def cleanNames(name):
    cleanname =name.lower().replace("'", "").replace(" ", "").replace(",", "").replace("-", "")
    return cleanname

In [ ]:
shanghai_most_recent_data['CleanUniNames'] = shanghai_most_recent_data.university_name.apply(cleanNames)

In [ ]:
times_most_recent_data['CleanUniNames'] = times_most_recent_data.university_name.apply(cleanNames)


In [ ]:
world_most_recent_data['CleanUniNames'] = world_most_recent_data.university_name.apply(cleanNames)

### Prendo solo le colone che mi interessano

In [ ]:
shanghai_sh=shanghai_most_recent_data[['CleanUniNames','university_name','world_rank']].rename(columns={"world_rank" : "world_rank_shanghai"})

In [ ]:
print(shanghai_sh)

In [ ]:
times_sh=times_most_recent_data[['CleanUniNames','world_rank']].rename(columns={"world_rank" : "world_rank_times"})

In [ ]:
world_sh=world_most_recent_data[['CleanUniNames','world_rank']].rename(columns={"world_rank" : "world_rank_world"})

### Inner join, prendo solo i nomi in comune

In [ ]:
JoinTableFirst = shanghai_sh.join(times_sh.set_index('CleanUniNames'), on='CleanUniNames', how='inner')

In [ ]:
JoinTable = JoinTableFirst.join(world_sh.set_index('CleanUniNames'), on='CleanUniNames', how='inner')

In [ ]:
JoinTable

### divido i range in due colone

In [ ]:
def splitRange(columnValue):
      result=columnValue.split('-')
      if len(result) >1 :
                return [int(result[0]),int(result[1])]
      else:
                return [int(result[0]),int(result[0])]

In [ ]:
Mylist =[]
def FindDiff(col1,col2,col3):
    
    Mylist.extend(splitRange(col1))
    Mylist.extend(splitRange(col2))
    Mylist.extend(splitRange(str(col3)))
    Mylist.sort()
    min1 = int(Mylist[0])
    max1 =int(Mylist[-1])
    Mylist.clear()
    return max1-min1

In [ ]:
JoinTable["MaxDiff"]=JoinTable.apply(lambda x: FindDiff(x.world_rank_shanghai, x.world_rank_times,x.world_rank_world), axis=1 )

In [ ]:
FinalTable = JoinTable[['university_name','world_rank_shanghai','world_rank_times', 'world_rank_world', 'MaxDiff']]

In [ ]:
FinalTable.head(5)


## 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [ ]:
times_most_recent_data.num_students

In [ ]:
times_most_recent_data.num_students.hasnans

#### Nota: vedo che la colona num_students usa la virgoa come seperatore di mille , percui gli tolgo

In [ ]:
times_most_recent_data['num_students']=times_most_recent_data.num_students.str.replace(',','').astype(float)

#### Controllo la collona female_male_ratio

In [ ]:
times_most_recent_data.female_male_ratio.hasnans

In [ ]:
times_most_recent_data.female_male_ratio.values

#### Vedo che ha valori  "-" e valori "nan" , percui gli devo ignorare

In [ ]:
times_most_recent_data.female_male_ratio

In [ ]:
times_most_recent_data['first_condition'] = times_most_recent_data['female_male_ratio'].str.contains('-')==True
times_most_recent_data['second_condition'] = times_most_recent_data['female_male_ratio'].isnull()
times_most_recent_data['third_condition'] = times_most_recent_data['num_students'].isnull()

### Costruisco due colone uno per numeri di maschi e uno per numero di femmine

In [ ]:
def CalculateFemale(row):
    if( row["first_condition"] | row["second_condition"] | row["third_condition"]):
        return None
    
    femaleratio = int(row["female_male_ratio"].split(':')[0])
    totalnrStudents = row["num_students"]
    return round((femaleratio/100) *totalnrStudents,0)
    

In [ ]:
times_most_recent_data["nr_female"]= times_most_recent_data.apply(CalculateFemale , axis=1)

In [ ]:
# Il numero di maschi è la differenza tra numero totale e numero di femmine
times_most_recent_data["nr_male"]=times_most_recent_data.num_students - times_most_recent_data.nr_female

In [ ]:
times_most_recent_data[["country","nr_female","nr_male","num_students","female_male_ratio"]]

In [ ]:
lista =["nr_female","nr_male","num_students"]

In [ ]:
# raggruppo per country e faccio la somma

female_male_df = times_most_recent_data.groupby('country', as_index=False)[lista].sum()

In [ ]:
female_male_df.tail(16)

### Note : Vedo che 2 nomi di universita sono scritti male percui gli elimino

In [ ]:
female_male_df.drop([67,71], inplace=True)

## 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [ ]:
def Calcolate_ratio(row):
    if math.isnan(row["nr_male"]) or math.isnan(row["nr_female"]) or row["nr_male"] == 0:
        return None
    else:
        return round((row["nr_female"]/row["nr_male"])*100)

In [ ]:
times_most_recent_data["Ratio"] = times_most_recent_data.apply(Calcolate_ratio,axis=1)

In [ ]:
times_most_recent_data.Ratio

In [ ]:
avarage_ratio = round(times_most_recent_data.Ratio.mean(),2)

In [ ]:
avarage_ratio

In [ ]:
female_male_below_ratio = times_most_recent_data[times_most_recent_data.Ratio < avarage_ratio]

In [ ]:
female_male_below_ratio[["university_name","Ratio"]]

## 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

In [ ]:
# Number of total student for country i have it from "female_male_df"
female_male_df.head()

In [ ]:
list_of_columns=["num_students","country"]
students_below_ratio=female_male_below_ratio.groupby('country', as_index=False)[list_of_columns].sum()

In [ ]:
ratio_below_and_total = pd.merge(female_male_df, students_below_ratio, how="inner", on=["country"])
ratio_below_and_total["Ratio"] = round((ratio_below_and_total.num_students_y/ratio_below_and_total.num_students_x)*100,2)

In [ ]:
#rename "num_students_x" to "num_students_total"  and "num_students_y" to "num_students_below"
ratio_below_and_total = ratio_below_and_total.rename(columns={'num_students_x': 'num_students_total', 'num_students_y': 'num_students_below'})

In [ ]:
ratio_below_and_total.head(10)

## 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [ ]:
educational_attainment_supplementary =readDataSet(educational_attainment_supplementary_data)

In [ ]:
index_to_drop=educational_attainment_supplementary[(educational_attainment_supplementary.country_name.isnull() ==True) | (educational_attainment_supplementary.series_name.isnull() ==True) ].index.values

In [ ]:
educational_attainment_supplementary.drop(index_to_drop, inplace=True)


## 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [ ]:
value_vars= ['1985', '1986', '1987', '1990', '1991',
       '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2015']
id_vars=['country_name', 'series_name']

In [ ]:
unpivot_attainment=pd.melt(educational_attainment_supplementary,id_vars=id_vars , value_vars=value_vars, var_name='year', value_name='value')

In [ ]:
unpivot_attainment.head(5)

## 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

### nota : normalizo i nomi di universita utilizando la funzione cleanNames

In [ ]:
times_df["uni_name_normalazed"] = times_df.university_name.apply(cleanNames)

In [ ]:
shanghai_df["uni_name_normalazed"]= shanghai_df.university_name.apply(cleanNames)

In [ ]:
world_df["uni_name_normalazed"] = world_df.university_name.apply(cleanNames)

In [ ]:
world_df.uni_name_normalazed

In [ ]:
### procedo a prendere i nomi di universta per ciascun dataframe

In [ ]:
times_df

In [ ]:
times_uni = times_df[["university_name","uni_name_normalazed"]].groupby("uni_name_normalazed").max()

In [ ]:
times_uni

In [ ]:
shanghai_uni = shanghai_df[["university_name","uni_name_normalazed"]].groupby("uni_name_normalazed").max()

In [ ]:
world_uni = world_df[["university_name","uni_name_normalazed"]].groupby("uni_name_normalazed").max()

In [ ]:
all_uni = times_uni.append(shanghai_uni).append(world_uni)

In [ ]:
all_uni_group=all_uni.groupby("uni_name_normalazed").count()

In [ ]:
#i want to take back normal university name so i make a join
all_uni_count_merge = pd.merge(all_uni,all_uni_group, on="uni_name_normalazed")

In [ ]:
# i have duplicates so i remove them, take off index, and rename all
all_uni_count=all_uni_count_merge.drop_duplicates().reset_index()\
                .drop("uni_name_normalazed" ,axis=1)\
                .rename(columns={'university_name_x': 'university_name','university_name_y':'count'}) 

In [ ]:
all_uni_count.head(5)

## 11. In the times ranking, compute the number of times each university appears

In [ ]:
times_count_uni=times_df.copy()

In [ ]:
times_count_uni_pc =times_count_uni[["university_name","uni_name_normalazed"]]\
                    .groupby("university_name" , as_index=False).count() \
                     .rename(columns={'uni_name_normalazed': 'count'}) 

In [ ]:
times_count_uni_pc.head(10)

## 12. Find the universities that appear at most twice in the times ranking.

In [ ]:
uni_most_twice = times_count_uni_pc[times_count_uni_pc['count'] >= 2]

In [ ]:
uni_most_twice.head(5)

## 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [ ]:
shanghai_no_range = shanghai_df[shanghai_df['world_rank'].str.contains('-')==False]

In [ ]:
times_no_range = times_df[times_df['world_rank'].str.contains('-')==False]

In [ ]:
shanghai_no_range =shanghai_no_range[["uni_name_normalazed","university_name","world_rank","year"]]

In [ ]:
shanghai_no_range.world_rank = shanghai_no_range.world_rank.astype('int')

In [ ]:
times_no_range=times_no_range[["uni_name_normalazed","university_name","world_rank","year"]]

In [ ]:
times_no_range.world_rank=times_no_range.world_rank.astype('int')

In [ ]:
worlddf=world_df[world_df["world_rank"].isnull() == False][["uni_name_normalazed","university_name","world_rank","year"]]

In [ ]:
first_union_df =pd.merge(shanghai_no_range,times_no_range,on=["uni_name_normalazed","year"])\
                            .drop("university_name_y",axis=1)\
                             .rename(columns={'university_name_x': 'university','world_rank_x':'shanghai_rank','world_rank_y':'times_rank'}) 

In [ ]:
first_union_df

In [ ]:
same_position = pd.merge(first_union_df,worlddf, on=["uni_name_normalazed","year"] )\
                        .drop(['university_name','uni_name_normalazed'],axis=1)

In [ ]:
result = same_position[(same_position['shanghai_rank'] == same_position['times_rank'])\
                              & (same_position['times_rank'] == same_position['world_rank']) ]

In [ ]:
result